<h2>LKH 2.0.9</h2>

**Note:** Notebook borrowed from Kagggle author `jsaguiar`

LKH is an effective implementation of the Lin-Kernighan heuristic and is considered one of the best algorithms available for the TSP problem. Even though the algorithm is approximate, optimal solutions are produced with an impressively high frequency. It has been implemented in the programming language C and is distributed for academic and non-commercial use.

In this notebook I will show how to run LKH for this competition in a kernel environment. For more information please refer to [the official page](http://akira.ruc.dk/~keld/research/LKH/) or the [original paper](http://akira.ruc.dk/~keld/research/LKH/LKH-2.0/DOC/LKH_REPORT.pdf). 

Notes:

* I'm not considering the prime penalty.
* Internet must be enabled.


Acknowledgments:  LKH author, [Keld Helsgaun](https://www.kaggle.com/keldhelsgaun).


<h2>2. Build LKH</h2>

In [2]:
%%bash -e
wget http://akira.ruc.dk/~keld/research/LKH/LKH-2.0.9.tgz
tar xvfz LKH-2.0.9.tgz
cd LKH-2.0.9
make

LKH-2.0.9/
LKH-2.0.9/pr2392.par
LKH-2.0.9/E3k.0.par
LKH-2.0.9/Makefile
LKH-2.0.9/xray14012_1.tsp
LKH-2.0.9/E3k.0.pi
LKH-2.0.9/E3k.0.tsp
LKH-2.0.9/pr2392.tsp
LKH-2.0.9/DOC/
LKH-2.0.9/README.txt
LKH-2.0.9/xray14012_1.par
LKH-2.0.9/SRC/
LKH-2.0.9/SRC/RestoreTour.c
LKH-2.0.9/SRC/SolveKMeansSubproblems.c
LKH-2.0.9/SRC/IsCommonEdge.c
LKH-2.0.9/SRC/ReadProblem.c
LKH-2.0.9/SRC/BestKOptMove.c
LKH-2.0.9/SRC/Distance_SPECIAL.c
LKH-2.0.9/SRC/CreateCandidateSet.c
LKH-2.0.9/SRC/OBJ/
LKH-2.0.9/SRC/Forbidden.c
LKH-2.0.9/SRC/Best5OptMove.c
LKH-2.0.9/SRC/RecordBetterTour.c
LKH-2.0.9/SRC/Best4OptMove.c
LKH-2.0.9/SRC/Exclude.c
LKH-2.0.9/SRC/C.c
LKH-2.0.9/SRC/IsCandidate.c
LKH-2.0.9/SRC/Make3OptMove.c
LKH-2.0.9/SRC/Make2OptMove.c
LKH-2.0.9/SRC/ResetCandidateSet.c
LKH-2.0.9/SRC/LKHmain.c
LKH-2.0.9/SRC/SolveSFCSubproblems.c
LKH-2.0.9/SRC/ERXT.c
LKH-2.0.9/SRC/fscanint.c
LKH-2.0.9/SRC/eprintf.c
LKH-2.0.9/SRC/Gain23.c
LKH-2.0.9/SRC/Heap.c
LKH-2.0.9/SRC/GetTime.c
LKH-2.0.9/SRC/SolveRoheSubproblems.c
LKH-2.0.9/SR

--2021-02-11 17:45:40--  http://akira.ruc.dk/~keld/research/LKH/LKH-2.0.9.tgz
Resolving akira.ruc.dk (akira.ruc.dk)... 130.225.220.230
Connecting to akira.ruc.dk (akira.ruc.dk)|130.225.220.230|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1505409 (1.4M) [application/x-gzip]
Saving to: ‘LKH-2.0.9.tgz’

     0K .......... .......... .......... .......... ..........  3% 95.0K 15s
    50K .......... .......... .......... .......... ..........  6%  189K 11s
   100K .......... .......... .......... .......... .......... 10% 3.13M 7s
   150K .......... .......... .......... .......... .......... 13% 11.4M 5s
   200K .......... .......... .......... .......... .......... 17% 93.7K 7s
   250K .......... .......... .......... .......... .......... 20% 35.4M 5s
   300K .......... .......... .......... .......... .......... 23% 72.4M 4s
   350K .......... .......... .......... .......... .......... 27% 79.9M 4s
   400K .......... .......... .......... .......... ........

<h2>3. Prepare inputs</h2>

We need to write a TSPLIB file and a parameters file to run LKH.

In [2]:
import utils
import collections
filename = "b_lovely_landscapes"


def create_data_model():
    """Stores the data for the problem."""
    alignments, tags = utils.read_input(filename)
    find_tag = collections.defaultdict(list)
    for idx, image_tags in enumerate(tags):
        for tag in image_tags:
            find_tag[tag].append(idx)
    pairs_set = set()
    for key, value in find_tag.items():
        if len(value) == 2:
            if value[0] > value[1]:
                value[1], value[0] = value[0], value[1]
            pairs_set.add((value[0], value[1]))
        elif len(value) > 2:
            raise ValueError("The same tag appears in %d photos, not expected in subpart-B" % len(value))
    return pairs_set, len(tags)

pairs, image_count = create_data_model()
print(f"Number of images {image_count} and number of pairs: {len(pairs)}")

Number of images 80000 and number of pairs: 200000


In [9]:
import tqdm

def write_tsp(image_count, filename):
    with open(filename, 'w') as f:
        f.write('NAME : %s\n' % filename)
        f.write('COMMENT : %s\n' % filename)
        f.write('TYPE : TSP\n')
        f.write('DIMENSION : %d\n' % image_count)
        f.write("EDGE_DATA_FORMAT: EDGE_LIST\n")
        f.write('EDGE_DATA_SECTION\n')

        c = 0
        for a in tqdm.trange(image_count):
            output = ""
            for b in range(a + 1, image_count):
                if (a, b) in pairs:
                    output += f'{a} {b}\n'
            c += 1
            if c == 20:
                f.write(output)
                c = 0

        f.write("-1\n")
        f.write('EOF\n')

image_count = 1000
tspfile = "images.tsp"
write_tsp(image_count, tspfile)

100%|██████████| 1000/1000 [00:00<00:00, 12704.70it/s]


For a full list of parameters check the [LKH User Guide](http://akira.ruc.dk/~keld/research/LKH/LKH-2.0/DOC/LKH-2.0_USER_GUIDE.pdf).  I've also created a [discussion here](https://www.kaggle.com/c/traveling-santa-2018-prime-paths/discussion/73694), since there are many options.

In [11]:
def write_parameters(parameters, filename='params.par'):
    with open(filename, 'w') as f:
        for param, value in parameters:
            f.write("{} = {}\n".format(param, value))
    print("Parameters saved as", filename)

parameters = [
    ("PROBLEM_FILE", tspfile),
    ("OUTPUT_TOUR_FILE", "tsp_solution.csv"),
    ("SEED", 2018),
    ('CANDIDATE_SET_TYPE', 'POPMUSIC'), #'NEAREST-NEIGHBOR', 'ALPHA'),
    ('INITIAL_PERIOD', 10000),
    ('MAX_TRIALS', 1000),
]
write_parameters(parameters)

Parameters saved as params.par


<h2>4. Run and write submission</h2>

I'm using a timeout to kill the process after 5 hours.

In [ ]:
%%bash -e
cd ./LKH-2.0.9
timeout 18000s ./LKH params.par

Read the output file and make a list (tour) of cities.

In [ ]:
def read_tour(filename):
    tour = []
    for line in open(filename).readlines():
        line = line.replace('\n', '')
        try:
            tour.append(int(line) - 1)
        except ValueError as e:
            pass  # skip if not a city id (int)
    return tour[:-1]

tour = read_tour('../working/LKH-2.0.9/tsp_solution.csv')
print("Tour length", len(tour))

Finally, we can print the score (considering prime twist) and write the submission file.

In [ ]:
import numpy as np
import sympy

def score_tour(tour):
    df = cities.reindex(tour + [0]).reset_index()
    primes = list(sympy.primerange(0, len(cities)))
    df['prime'] = df.CityId.isin(primes).astype(int)
    df['dist'] = np.hypot(df.X - df.X.shift(-1), df.Y - df.Y.shift(-1))
    df['penalty'] = df['dist'][9::10] * (1 - df['prime'][9::10]) * 0.1
    return df.dist.sum() + df.penalty.sum()

def write_submission(tour, filename):
    assert set(tour) == set(range(len(tour)))
    pd.DataFrame({'Path': list(tour) + [0]}).to_csv(filename, index=False)

print("Final score", score_tour(tour))
write_submission(tour, 'submission.csv')

Thanks for reading and merry christmas!